In [ ]:
!curl -LO https://github.com/chsponciano/pocket-parkinson-dataset/raw/main/dataset.zip
!unzip -qq dataset.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   153  100   153    0     0    508      0 --:--:-- --:--:-- --:--:--   510
100 6958k  100 6958k    0     0  7916k      0 --:--:-- --:--:-- --:--:-- 7916k


In [ ]:
import os
import numpy as np
import keras

In [ ]:
_rotation = 20 
_zoom = 0.5
_flip = True
_batch_size = 32
_random_state = 42
_input_shape = (150, 150, 3)
_dropout = 0.6
_alpha = 1e-5
_epochs = 10
_hidden_unit = 32
_kernel_size = 3
_trainable_layer = 'block5_conv1'

# Data preparation

In [ ]:
from PIL import Image

def select_image(filename):
    _image = Image.open(filename)
    _image = _image.convert('RGB')
    _image = _image.resize((150,150))
    return np.asarray(_image)

In [ ]:
def load_class(directory, label_class, images, labels):
    for filename in os.listdir(directory):
        _path = directory + filename
        images.append(select_image(_path))
        labels.append(label_class)
    return images, labels

In [ ]:
def select_dataset(directory):
    _images = list()
    _labels = list()
    for subdir in os.listdir(directory):
        _path = directory + subdir + '/'
        if not os.path.isdir(_path):
            continue
        _images, _labels = load_class(_path, subdir, _images, _labels)
    return _images, _labels

In [ ]:
_images, _labels = select_dataset('./data/')
print(len(_images), len(_labels))

5220 5220


In [ ]:
from sklearn.preprocessing import LabelBinarizer

_labelBinarizer = LabelBinarizer()

In [ ]:
from keras.utils import to_categorical

def normalize(images, labels):
    images = np.array(images) / 255.0
    labels = np.array(labels)
    labels = _labelBinarizer.fit_transform(labels)
    labels = to_categorical(labels)
    return images, labels

_images, _labels = normalize(_images, _labels)

In [ ]:
from sklearn.model_selection import train_test_split

_x_train, _x_test, _y_train, _y_test = train_test_split(_images, _labels, test_size=0.20, stratify=_labels, random_state=_random_state)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

_train_data_augmentation = ImageDataGenerator(rotation_range=_rotation, zoom_range=_zoom, horizontal_flip=_flip)
_train_data_augmentation.fit(_x_train)
_data_augmentation = _train_data_augmentation.flow(_x_train, _y_train, batch_size=_batch_size)

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

def _get_callbacks(model_name, verbose=1):
    _checkpoint = ModelCheckpoint(os.getcwd() + f'{model_name}.hdf5', monitor='val_acc', save_best_only=True, mode='max', verbose=verbose)
    _plateau = ReduceLROnPlateau(monitor='val_acc', factor=0.1, min_delta=_alpha, patience=5, verbose=verbose)
    return [_checkpoint, _plateau]

# LSTM [Horchreiter et al., 1997]

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, BatchNormalization, Dense, Dropout, Flatten, LSTM, RepeatVector, TimeDistributed, LSTM

# https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8502031 - page 8

_model = Sequential()

# layer 1-5
_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same' , activation ='relu', input_shape =_input_shape))
_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
_model.add(Dropout(0.25))

# layer 6 - 10
_model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
_model.add(Dropout(0.25))

# layer 11 - 15
_model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(1,2), strides=(1,2)))
_model.add(Dropout(0.25))

# layer 16 - 18 
_model.add(Dense(256 , activation ='relu'))
_model.add(Dropout(0.25))

# layer 19
_model.add(TimeDistributed(Flatten()))
_model.add(LSTM(32, return_sequences=False))

# layer 20 - 22
_model.add(Dense(2, activation = "softmax"))

_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 150, 150, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 128)       1

In [ ]:
_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
_callbacks = _get_callbacks('/vgg19-lstm-2')
_history = _model.fit_generator(_data_augmentation,
                                epochs = _epochs, 
                                validation_data = (_x_train, _y_train),
                                verbose = 1, 
                                steps_per_epoch=_x_train.shape[0] // _batch_size, 
                                callbacks=_callbacks)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
130/130 [==============================] - 2201s 17s/step - loss: 0.6564 - acc: 0.6214 - val_loss: 0.6165 - val_acc: 0.7134

Epoch 00001: val_acc improved from -inf to 0.71336, saving model to /content/vgg19-lstm-2.hdf5
Epoch 2/10
130/130 [==============================] - 2220s 17s/step - loss: 0.6068 - acc: 0.6998 - val_loss: 0.6127 - val_acc: 0.6537

Epoch 00002: val_acc did not improve from 0.71336
Epoch 3/10
130/130 [==============================] - 2238s 17s/step - loss: 0.5872 - acc: 0.7208 - val_loss: 0.5398 - val_acc: 0.7349

Epoch 00003: val_acc improved from 0.71336 to 0.73491, saving model to /content/vgg19-lstm-2.hdf5
Epoch 4/10
130/130 [==============================] - 2240s 17s/step - loss: 0.5607 - acc: 0.7315 - val_loss: 0.4781 - val_acc: 0.8147

Epoch 00004: val_acc improved from 0.73491 to 0.81466, saving model to /content/vgg19-lstm-2.hdf5
Epoch 5/10
130/130 [==============================] - 2241s 17s/step - loss: 0.4878 - acc: 0.7915 - val_loss: 0.907

In [ ]:
_model.evaluate(_x_train, _y_train)

131/131 [==============================] - 486s 4s/step - loss: 0.1332 - acc: 0.9734


[0.13318341970443726, 0.9734195470809937]

# LSTM-LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, BatchNormalization, Dense, Dropout, Flatten, LSTM, RepeatVector, TimeDistributed

# https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8502031 - page 8

_model = Sequential()

# layer 1-5
_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same' , activation ='relu', input_shape =_input_shape))
_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
_model.add(Dropout(0.25))

# layer 6 - 10
_model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
_model.add(Dropout(0.25))

# layer 11 - 15
_model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(1,2), strides=(1,2)))
_model.add(Dropout(0.25))

# layer 16 - 18 
_model.add(Dense(256 , activation ='relu'))
_model.add(Dropout(0.25))

# layer 19
_model.add(TimeDistributed(Flatten()))
_model.add(LSTM(64, return_sequences=True))
_model.add(LSTM(32, return_sequences=False))

# layer 20 - 22
_model.add(Dense(2, activation = "softmax"))

_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 150, 150, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 128)       1

In [ ]:
_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
_callbacks = _get_callbacks('/vgg19-lstm-3')
_history = _model.fit_generator(_data_augmentation,
                                epochs = _epochs, 
                                validation_data = (_x_train, _y_train),
                                verbose = 1, 
                                steps_per_epoch=_x_train.shape[0] // _batch_size, 
                                callbacks=_callbacks)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
130/130 [==============================] - 1604s 12s/step - loss: 0.6019 - acc: 0.6862 - val_loss: 0.6229 - val_acc: 0.6667

Epoch 00001: val_acc improved from -inf to 0.66667, saving model to /content/vgg19-lstm-3.hdf5
Epoch 2/10
130/130 [==============================] - 1620s 12s/step - loss: 0.4065 - acc: 0.8253 - val_loss: 1.3873 - val_acc: 0.5000

Epoch 00002: val_acc did not improve from 0.66667
Epoch 3/10
130/130 [==============================] - 1621s 12s/step - loss: 0.3112 - acc: 0.8745 - val_loss: 0.2888 - val_acc: 0.9028

Epoch 00003: val_acc improved from 0.66667 to 0.90278, saving model to /content/vgg19-lstm-3.hdf5
Epoch 4/10
130/130 [==============================] - 1626s 13s/step - loss: 0.2198 - acc: 0.9170 - val_loss: 0.1196 - val_acc: 0.9547

Epoch 00004: val_acc improved from 0.90278 to 0.95474, saving model to /content/vgg19-lstm-3.hdf5
Epoch 5/10
130/130 [==============================] - 1627s 13s/step - loss: 0.1369 - acc: 0.9503 - val_loss: 0.029

In [ ]:
_model.evaluate(_x_train, _y_train)

131/131 [==============================] - 290s 2s/step - loss: 0.0462 - acc: 0.9959


[0.046178679913282394, 0.9959291219711304]

# GRU [Cho et al., 2014]
Combina o input e forget gate no update gate

Mais usando em casos binarios (azul ou amarelo)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, BatchNormalization, Dense, Dropout, Flatten, GRU, RepeatVector, TimeDistributed

# https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8502031 - page 8

_model = Sequential()

# layer 1-5
_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same' , activation ='relu', input_shape =_input_shape))
_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
_model.add(Dropout(0.25))

# layer 6 - 10
_model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
_model.add(Dropout(0.25))

# layer 11 - 15
_model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(1,2), strides=(1,2)))
_model.add(Dropout(0.25))

# layer 16 - 18 
_model.add(Dense(256 , activation ='relu'))
_model.add(Dropout(0.25))

# layer 19
_model.add(TimeDistributed(Flatten()))
_model.add(GRU(32, return_sequences=False))

# layer 20 - 22
_model.add(Dense(2, activation = "softmax"))

_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 150, 150, 64)      1792      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
batch_normalization_3 (Batch (None, 150, 150, 64)      256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 75, 75, 128)      

In [ ]:
_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
_callbacks = _get_callbacks('/vgg19-lstm-4')
_history = _model.fit_generator(_data_augmentation,
                                epochs = _epochs, 
                                validation_data = (_x_train, _y_train),
                                verbose = 1, 
                                steps_per_epoch=_x_train.shape[0] // _batch_size, 
                                callbacks=_callbacks)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
130/130 [==============================] - 1613s 12s/step - loss: 0.6460 - acc: 0.6557 - val_loss: 0.7511 - val_acc: 0.5000

Epoch 00001: val_acc improved from -inf to 0.50000, saving model to /content/vgg19-lstm-4.hdf5
Epoch 2/10
130/130 [==============================] - 1616s 12s/step - loss: 0.5550 - acc: 0.7324 - val_loss: 1.2683 - val_acc: 0.5000

Epoch 00002: val_acc did not improve from 0.50000
Epoch 3/10
130/130 [==============================] - 1612s 12s/step - loss: 0.3044 - acc: 0.8898 - val_loss: 1.2991 - val_acc: 0.5000

Epoch 00003: val_acc did not improve from 0.50000
Epoch 4/10
130/130 [==============================] - 1610s 12s/step - loss: 0.2069 - acc: 0.9270 - val_loss: 1.6001 - val_acc: 0.5000

Epoch 00004: val_acc did not improve from 0.50000
Epoch 5/10
130/130 [==============================] - 1609s 12s/step - loss: 0.1555 - acc: 0.9401 - val_loss: 0.8448 - val_acc: 0.7584

Epoch 00005: val_acc improved from 0.50000 to 0.75838, saving model to /con

In [ ]:
_model.evaluate(_x_train, _y_train)

131/131 [==============================] - 297s 2s/step - loss: 0.0177 - acc: 0.9952


[0.017689209431409836, 0.9952107071876526]

# GRU-GRU

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, BatchNormalization, Dense, Dropout, Flatten, GRU, RepeatVector, TimeDistributed

# https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8502031 - page 8

_model = Sequential()

# layer 1-5
_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same' , activation ='relu', input_shape =_input_shape))
_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
_model.add(Dropout(0.25))

# layer 6 - 10
_model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
_model.add(Dropout(0.25))

# layer 11 - 15
_model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(1,2), strides=(1,2)))
_model.add(Dropout(0.25))

# layer 16 - 18 
_model.add(Dense(256 , activation ='relu'))
_model.add(Dropout(0.25))

# layer 19
_model.add(TimeDistributed(Flatten()))
_model.add(GRU(64, return_sequences=True))
_model.add(GRU(32, return_sequences=False))

# layer 20 - 22
_model.add(Dense(2, activation = "softmax"))

_model.summary()

In [ ]:
_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
_callbacks = _get_callbacks('/vgg19-lstm-5')
_history = _model.fit_generator(_data_augmentation,
                                epochs = _epochs, 
                                validation_data = (_x_train, _y_train),
                                verbose = 1, 
                                steps_per_epoch=_x_train.shape[0] // _batch_size, 
                                callbacks=_callbacks)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
130/130 [==============================] - 2405s 18s/step - loss: 0.6350 - acc: 0.6643 - val_loss: 1.1058 - val_acc: 0.5000

Epoch 00001: val_acc improved from -inf to 0.50000, saving model to /content/vgg19-lstm-5.hdf5
Epoch 2/10
130/130 [==============================] - 2429s 19s/step - loss: 0.2857 - acc: 0.8903 - val_loss: 2.1045 - val_acc: 0.5000

Epoch 00002: val_acc did not improve from 0.50000
Epoch 3/10
130/130 [==============================] - 2413s 19s/step - loss: 0.1340 - acc: 0.9537 - val_loss: 2.4706 - val_acc: 0.5000

Epoch 00003: val_acc did not improve from 0.50000
Epoch 4/10
130/130 [==============================] - 2413s 19s/step - loss: 0.1075 - acc: 0.9599 - val_loss: 0.1435 - val_acc: 0.9514

Epoch 00004: val_acc improved from 0.50000 to 0.95139, saving model to /content/vgg19-lstm-5.hdf5
Epoch 5/10
130/130 [==============================] - 2408s 19s/step - loss: 0.0686 - acc: 0.9776 - val_loss: 2.4219 - val_acc: 0.5084

Epoch 00005: val_acc did no

In [ ]:
_model.evaluate(_x_train, _y_train)

131/131 [==============================] - 493s 4s/step - loss: 0.0074 - acc: 0.9990


[0.007397003937512636, 0.9990421533584595]

# BiLSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, BatchNormalization, Dense, Dropout, Flatten, LSTM, RepeatVector, TimeDistributed, Bidirectional, LSTM

# https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8502031 - page 8

_model = Sequential()

# layer 1-5
_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same' , activation ='relu', input_shape =_input_shape))
_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
_model.add(Dropout(0.25))

# layer 6 - 10
_model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
_model.add(Dropout(0.25))

# layer 11 - 15
_model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(1,2), strides=(1,2)))
_model.add(Dropout(0.25))

# layer 16 - 18 
_model.add(Dense(256 , activation ='relu'))
_model.add(Dropout(0.25))

# layer 19
_model.add(TimeDistributed(Flatten()))
_model.add(Bidirectional(LSTM(32, return_sequences=False)))

# layer 20 - 22
_model.add(Dense(2, activation = "softmax"))

_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 150, 150, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 128)       1

In [ ]:
_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
_callbacks = _get_callbacks('/vgg19-lstm-6')
_history = _model.fit_generator(_data_augmentation,
                                epochs = _epochs, 
                                validation_data = (_x_train, _y_train),
                                verbose = 1, 
                                steps_per_epoch=_x_train.shape[0] // _batch_size, 
                                callbacks=_callbacks)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
130/130 [==============================] - 2482s 19s/step - loss: 0.6435 - acc: 0.6435 - val_loss: 0.7645 - val_acc: 0.5000

Epoch 00001: val_acc improved from -inf to 0.50000, saving model to /content/vgg19-lstm-6.hdf5
Epoch 2/10
130/130 [==============================] - 2514s 19s/step - loss: 0.6486 - acc: 0.6405 - val_loss: 0.6193 - val_acc: 0.8194

Epoch 00002: val_acc improved from 0.50000 to 0.81944, saving model to /content/vgg19-lstm-6.hdf5
Epoch 3/10
130/130 [==============================] - 2520s 19s/step - loss: 0.5096 - acc: 0.8129 - val_loss: 0.9383 - val_acc: 0.5000

Epoch 00003: val_acc did not improve from 0.81944
Epoch 4/10
130/130 [==============================] - 2525s 19s/step - loss: 0.2820 - acc: 0.9091 - val_loss: 0.6715 - val_acc: 0.5745

Epoch 00004: val_acc did not improve from 0.81944
Epoch 5/10
130/130 [==============================] - 2540s 20s/step - loss: 0.1906 - acc: 0.9383 - val_loss: 1.1252 - val_acc: 0.5007

Epoch 00005: val_acc did no

In [ ]:
_model.evaluate(_x_train, _y_train)

# BiGRU

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, BatchNormalization, Dense, Dropout, Flatten, LSTM, RepeatVector, TimeDistributed, Bidirectional, GRU

# https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8502031 - page 8

_model = Sequential()

# layer 1-5
_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same' , activation ='relu', input_shape =_input_shape))
_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
_model.add(Dropout(0.25))

# layer 6 - 10
_model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
_model.add(Dropout(0.25))

# layer 11 - 15
_model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' , activation ='relu'))
_model.add(BatchNormalization())
_model.add(MaxPool2D(pool_size=(1,2), strides=(1,2)))
_model.add(Dropout(0.25))

# layer 16 - 18 
_model.add(Dense(256 , activation ='relu'))
_model.add(Dropout(0.25))

# layer 19
_model.add(TimeDistributed(Flatten()))
_model.add(Bidirectional(GRU(32, return_sequences=False)))

# layer 20 - 22
_model.add(Dense(2, activation = "softmax"))

_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 150, 150, 64)      1792      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
batch_normalization_4 (Batch (None, 150, 150, 64)      256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 75, 75, 128)      

In [ ]:
_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
_callbacks = _get_callbacks('/vgg19-lstm-1')
_history = _model.fit_generator(_data_augmentation,
                                epochs = _epochs, 
                                validation_data = (_x_train, _y_train),
                                verbose = 1, 
                                steps_per_epoch=_x_train.shape[0] // _batch_size, 
                                callbacks=_callbacks)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
130/130 [==============================] - 2162s 17s/step - loss: 0.6214 - acc: 0.6769 - val_loss: 0.9615 - val_acc: 0.5005

Epoch 00001: val_acc improved from -inf to 0.50048, saving model to /content/vgg19-lstm-1.hdf5
Epoch 2/10
130/130 [==============================] - 2165s 17s/step - loss: 0.2827 - acc: 0.9013 - val_loss: 1.1465 - val_acc: 0.5002

Epoch 00002: val_acc did not improve from 0.50048
Epoch 3/10
130/130 [==============================] - 2176s 17s/step - loss: 0.1893 - acc: 0.9416 - val_loss: 0.8520 - val_acc: 0.7028

Epoch 00003: val_acc improved from 0.50048 to 0.70283, saving model to /content/vgg19-lstm-1.hdf5
Epoch 4/10
130/130 [==============================] - 2180s 17s/step - loss: 0.1444 - acc: 0.9521 - val_loss: 1.6817 - val_acc: 0.5419

Epoch 00004: val_acc did not improve from 0.70283
Epoch 5/10
130/130 [==============================] - 2176s 17s/step - loss: 0.1389 - acc: 0.9517 - val_loss: 0.2397 - val_acc: 0.9059

Epoch 00005: val_acc improv

In [ ]:
_model.evaluate(_x_train, _y_train)

131/131 [==============================] - 461s 4s/step - loss: 0.0214 - acc: 0.9945


[0.021358083933591843, 0.9944923520088196]

In [ ]:
_parkinson_image = select_image('/content/data/parkinson/img_1_1.png')
_parkinson_image = np.array([_parkinson_image]) / 255.0
_model.predict(_parkinson_image)

array([[7.8957271e-07, 9.9999917e-01]], dtype=float32)

In [ ]:
_normal_image = select_image('/content/data/normal/001-cl-01-090-046.png')
_normal_image = np.array([_normal_image]) / 255.0
_model.predict(_normal_image)

array([[9.9919122e-01, 8.0880243e-04]], dtype=float32)